In [7]:
import requests
import copy
import json
import math
from pprint import pprint

In [2]:
class Node(object):
  
    def __init__(self, parent=None, moves=None, state=None):
        self.parent = parent
        self.moves = moves
        self.state = state

        self.function = 0
        self.degree = 0
        self.heuristic = 0
    
    def calculate_function(self):
        return self.degree * self.heuristic

    def __eq__(self, other):
        return self.state == other

In [3]:
class Maze(object):
  
    def __init__(self, start_position, end_position, map_, height, length, endpoint):
        self.start_position = start_position[::-1]
        self.end_position = end_position[::-1]
        self.map_ = map_
        self.height = height
        self.length = length

        self.endpoint = endpoint

In [4]:
def possible_moves(node, maze):
    possible_nodes = list()

    current_position = node.moves[-1]

    left_x, up_x, right_x, down_x = current_position[0], current_position[0] - 1, current_position[0], current_position[0] + 1
    left_y, up_y, right_y, down_y = current_position[1] - 1, current_position[1], current_position[1] + 1, current_position[1]

    if up_x >= 0 and up_x < maze.height:
        if maze.map_[up_x][up_y] != 'X':
            if [up_x, up_y] not in node.moves:
                new_moves = copy.deepcopy(node.moves)
                new_moves.append([up_x, up_y])

                new_node = Node(node, new_moves, node.state + 'N')

                new_node.degree = node.degree + 1
                new_node.heuristic = calculate_heuristic([up_x, up_y], maze.end_position)

                new_node.function = new_node.calculate_function()

                possible_nodes.append(new_node)

    if left_y >= 0 and left_y < maze.length:
        if maze.map_[left_x][left_y] != 'X':
            if [left_x, left_y] not in node.moves:
                new_moves = copy.deepcopy(node.moves)
                new_moves.append([left_x, left_y])

                new_node = Node(node, new_moves, node.state + 'W')

                new_node.degree = node.degree + 1
                new_node.heuristic = calculate_heuristic([left_x, left_y], maze.end_position)

                new_node.function = new_node.calculate_function()

                possible_nodes.append(new_node)

    if right_y >= 0 and right_y < maze.length:
        if maze.map_[right_x][right_y] != 'X':
            if [right_x, right_y] not in node.moves:
                new_moves = copy.deepcopy(node.moves)
                new_moves.append([right_x, right_y])

                new_node = Node(node, new_moves, node.state + 'E')

                new_node.degree = node.degree + 1
                new_node.heuristic = calculate_heuristic([right_x, right_y], maze.end_position)

                new_node.function = new_node.calculate_function()

                possible_nodes.append(new_node)

    if down_x >= 0 and down_x < maze.height:
        if maze.map_[down_x][down_y] != 'X':
            if [down_x, down_y] not in node.moves:
                new_moves = copy.deepcopy(node.moves)
                new_moves.append([down_x, down_y])

                new_node = Node(node, new_moves, node.state + 'S')

                new_node.degree = node.degree + 1
                new_node.heuristic = calculate_heuristic([down_x, down_y], maze.end_position)

                new_node.function = new_node.calculate_function()

                possible_nodes.append(new_node)

    return possible_nodes

In [8]:
def calculate_heuristic(current_position, end_position):
    x = 0
    y = 0

    x = end_position[0] - current_position[0]
    y = end_position[1] - current_position[1]

    return math.sqrt((x*x) + (y*y))

In [12]:
def a_star(maze):
    visited_nodes = list()
    visitable_nodes = list()

    initial_node = Node(None, [maze.start_position], '')

    visitable_nodes.append(initial_node)

    while len(visitable_nodes) > 0:
        current_node = visitable_nodes[0]

        visitable_nodes.remove(current_node)
        visited_nodes.append(current_node)

        if current_node.moves[-1] == maze.end_position:
            return current_node

        possible_nodes = possible_moves(current_node, maze)

        if possible_nodes:
            for possible_node in possible_nodes:
                insertion_index = len(visitable_nodes)

                for index, node in enumerate(visitable_nodes):
                    if possible_node.function < node.function:
                        insertion_index = index
                        break

                visitable_nodes.insert(insertion_index, possible_node)

In [13]:
def post_solution(solution):
    url = 'https://api.noopschallenge.com' + maze.endpoint

    data = {
      "directions": result.state
    }

    final = requests.post(url, json.dumps(data))

    pprint(final.text)

In [17]:
req = requests.get("https://api.noopschallenge.com/mazebot/random?minSize=10&maxSize=20")

response = json.loads(req.text)

maze = Maze(response['startingPosition'], response['endingPosition'], response['map'], len(response['map']), len(response['map'][0]), response['mazePath'])

result = a_star(maze)

post_solution(result)

('{"result":"success","message":"You solved it in 0.813 seconds with 32 steps, '
 'the shortest possible '
 'solution.","shortestSolutionLength":32,"yourSolutionLength":32,"elapsed":813}')


In [18]:
print(maze.height, maze.length)

10 10
